### Having_ip_address

In [1]:
def having_ip_address(url):
  import re
  x = re.search('^(http|https)://\d+\.\d+\.\d+\.\d+\.*', url)
  if x:
    return 1
  else:
    return -1


In [2]:
having_ip_address('https://128.36.54.192/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')


1

### URL Length

In [3]:
def URL_Length(url):
  if len(url) < 54:
    return -1
  elif len(url) >= 54 and len(url) <=75:
    return 0
  else:
    return 1

In [4]:
URL_Length('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

1

### Having @ Symbol

In [5]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = -1    
  return at

In [6]:
haveAtSign('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

-1

### Prefix_Suffix  

In [7]:
def prefixSuffix(url):
    from urllib.parse import urlparse
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return -1

In [8]:
prefixSuffix('https://dfhdfhdfgs.tokyo/ja-jp/account/login')

-1

### Having sub domain

In [9]:
def sub_domain_count(url):
  !pip install tld
  from urllib.parse import urlparse
  from tld import get_tld
  domain = urlparse(url).netloc
  domain = domain.split('.')
  top_domain = get_tld(url)
  top_domain = top_domain.split('.')
  sub_domain = set(domain) - set(top_domain)
  count = len(sub_domain)
  if(count == 1):
    return -1
  if(count == 2):
    return 0
  else:
    return 1 

In [10]:
sub_domain_count('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

-1

###SSL final state

In [11]:
def sslVerify(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from urllib.request import Request, urlopen, ssl, socket
  import json
  from datetime import datetime
  split_url = urlsplit(url)
  #some site without http/https in the path
  port = '443'

  hostname = split_url.netloc
  context = ssl.create_default_context()
  try:
    with socket.create_connection((hostname, port)) as sock:
      with context.wrap_socket(sock, server_hostname=hostname) as ssock:
          data = json.dumps(ssock.getpeercert())
          res = json.loads(data)
    notBefore = datetime.strptime(res["notBefore"],'%b  %d %H:%M:%S %Y %Z').date()
    notAfter = datetime.strptime(res["notAfter"],'%b  %d %H:%M:%S %Y %Z').date()
    # print(notBefore, notAfter)
    if(ssl.SSLCertVerificationError(ssock) == True):
      return 1
    elif (notAfter.year-notBefore.year)+(notAfter.month-notBefore.month)*0.1 >= 1:
      return -1
    else:
      return 0
  except:
    return 1

In [12]:
sslVerify("http://postdebanks.com/DIE/POST/diepost/")

1

### Port

In [51]:
import requests
import json

def port(domain):
    try:
        response = requests.get("https://api.viewdns.info/portscan/?host="+domain+"&apikey=1bf03196763a201bcc66a59bf88ed8ddf7a9432f&output=json")
        response.raise_for_status()  # Raise an exception for any HTTP errors

        # Check if response contains valid JSON
        try:
            myjson = response.json()
        except json.JSONDecodeError as e:
            print(f"Failed to decode JSON response: {e}")
            return -1

        pref_stat = {21: 'closed', 22: 'closed', 23: 'closed', 80: 'open', 443: 'open', 445: 'closed', 1433: 'closed', 1521: 'closed', 3306: 'closed', 3389: 'closed'}

        flag = -1
        for port_data in myjson.get('response', {}).get('port', []):
            port_number = int(port_data.get('number', 0))
            port_status = port_data.get('status', '')

            if port_number in pref_stat and port_status != pref_stat[port_number]:
                flag = 1
                break  # Exit loop as soon as a port status doesn't match
        return flag
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return -1  # Return -1 in case of any request exception




In [52]:
# Test the function
print(port('postdebanks.com'))

Failed to decode JSON response: Expecting value: line 1 column 1 (char 0)
-1


### Request_URL

In [15]:
def request_url(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        print('Yes')
        content = page.content
        hostname, domain, path = get_domain(url)
  else:
        print('No state')
  
  Media = {'internals':[], 'externals':[], 'null':[]}

  def external_media(Media):
    total = len(Media['internals']) + len(Media['externals'])
    externals = len(Media['externals'])
    try:
        percentile = externals / float(total) * 100
    except:
        return 0
    return percentile
  
  def findMedia(Media,domain, hostname ):
    Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
    soup = BeautifulSoup(content, 'html.parser', from_encoding='iso-8859-1')
    for img in soup.find_all('img', src=True):
          dots = [x.start(0) for x in re.finditer('\.', img['src'])]
          if hostname in img['src'] or domain in img['src'] or len(dots) == 1 or not img['src'].startswith('http'):
              if not img['src'].startswith('http'):
                  if not img['src'].startswith('/'):
                      Media['internals'].append(hostname+'/'+img['src']) 
                  elif img['src'] in Null_format:
                      Media['null'].append(img['src'])  
                  else:
                      Media['internals'].append(hostname+img['src'])   
          else:
              Media['externals'].append(img['src'])
    for audio in soup.find_all('audio', src=True):
      dots = [x.start(0) for x in re.finditer('\.', audio['src'])]
      if hostname in audio['src'] or domain in audio['src'] or len(dots) == 1 or not audio['src'].startswith('http'):
          if not audio['src'].startswith('http'):
              if not audio['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+audio['src']) 
              elif audio['src'] in Null_format:
                  Media['null'].append(audio['src'])  
              else:
                  Media['internals'].append(hostname+audio['src'])   
      else:
          Media['externals'].append(audio['src'])
          
    for embed in soup.find_all('embed', src=True):
      dots = [x.start(0) for x in re.finditer('\.', embed['src'])]
      if hostname in embed['src'] or domain in embed['src'] or len(dots) == 1 or not embed['src'].startswith('http'):
          if not embed['src'].startswith('http'):
              if not embed['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+embed['src']) 
              elif embed['src'] in Null_format:
                  Media['null'].append(embed['src'])  
              else:
                  Media['internals'].append(hostname+embed['src'])   
      else:
          Media['externals'].append(embed['src'])
        
    for i_frame in soup.find_all('iframe', src=True):
      dots = [x.start(0) for x in re.finditer('\.', i_frame['src'])]
      if hostname in i_frame['src'] or domain in i_frame['src'] or len(dots) == 1 or not i_frame['src'].startswith('http'):
          if not i_frame['src'].startswith('http'):
              if not i_frame['src'].startswith('/'):
                  Media['internals'].append(hostname+'/'+i_frame['src']) 
              elif i_frame['src'] in Null_format:
                  Media['null'].append(i_frame['src'])  
              else:
                  Media['internals'].append(hostname+i_frame['src'])   
      else: 
          Media['externals'].append(i_frame['src'])
  
  findMedia(Media, domain, hostname)
  if external_media(Media) < 22:
    return -1
  elif external_media(Media) >= 22 and external_media(Media) < 61:
    return 0
  else:
    return 1

In [16]:
# request_url('http://www.budgetbots.com/server.php/Server%20update/index.php?email=USER@DOMAIN.com')

### Safe anchors

In [17]:
def url_anchor(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
    
  Anchor = {'safe':[], 'unsafe':[], 'null':[]}

  def anchor(Anchor, domain, hostname):
      soup = BeautifulSoup(content, 'html.parser', from_encoding='iso-8859-1')
      for href in soup.find_all('a', href=True):
        dots = [x.start(0) for x in re.finditer('\.', href['href'])]
        if hostname in href['href'] or domain in href['href'] or len(dots) == 1 or not href['href'].startswith('http'):
            if "#" in href['href'] or "javascript" in href['href'].lower() or "mailto" in href['href'].lower():
                 Anchor['unsafe'].append(href['href'])
        else:
            Anchor['safe'].append(href['href'])

  anchor(Anchor, domain, url)

  def safe_anchor(Anchor):
      total = len(Anchor['safe']) +  len(Anchor['unsafe'])
      unsafe = len(Anchor['unsafe'])
      try:
        percentile = unsafe / float(total) * 100
      except:
        return 0
      return percentile
  #print(safe_anchor(Anchor))   
  if safe_anchor(Anchor) < 31:
       return -1
  elif safe_anchor(Anchor) >= 31 and safe_anchor(Anchor) <= 67:
       return 0
  else:
       return 1


In [18]:
url_anchor('https://www.xiaoji.com/')

-1

### Links in Tags

In [19]:
def links_tag(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
  
  Link = {'internals':[], 'externals':[], 'null':[]}

  def find_links(Link, domain, hostname):
    soup = BeautifulSoup(content, 'html.parser', from_encoding='iso-8859-1')
    for link in soup.findAll('link', href=True):
        dots = [x.start(0) for x in re.finditer('\.', link['href'])]
        if hostname in link['href'] or domain in link['href'] or len(dots) == 1 or not link['href'].startswith('http'):
            if not link['href'].startswith('http'):
                if not link['href'].startswith('/'):
                    Link['internals'].append(hostname+'/'+link['href']) 
                elif link['href'] in Null_format:
                    Link['null'].append(link['href'])  
                else:
                    Link['internals'].append(hostname+link['href'])   
        else:
            Link['externals'].append(link['href'])

    for script in soup.find_all('script', src=True):
        dots = [x.start(0) for x in re.finditer('\.', script['src'])]
        if hostname in script['src'] or domain in script['src'] or len(dots) == 1 or not script['src'].startswith('http'):
            if not script['src'].startswith('http'):
                if not script['src'].startswith('/'):
                    Link['internals'].append(hostname+'/'+script['src']) 
                elif script['src'] in Null_format:
                    Link['null'].append(script['src'])  
                else:
                    Link['internals'].append(hostname+script['src'])   
        else:
            Link['externals'].append(link['href'])
  
  def links_in_tags(Link):
    total = len(Link['internals']) +  len(Link['externals'])
    internals = len(Link['internals'])
    try:
        percentile = internals / float(total) * 100
    except:
        return 0
    return percentile
  #print(links_in_tags(Link))
  if links_in_tags(Link) < 17:
    return -1
  elif links_in_tags(Link) >= 17 and links_in_tags(Link) <= 81:
    return 0
  else:
    return 1

In [20]:
links_tag('https://www.xiaoji.com/')

-1

###SFH

In [21]:
def sfh(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
  
  Form = {'internals':[], 'externals':[], 'null':[]}

  def findForm(Form, domain, hostname):
    for form in soup.findAll('form', action=True):
      dots = [x.start(0) for x in re.finditer('\.', form['action'])]
      if hostname in form['action'] or domain in form['action'] or len(dots) == 1 or not form['action'].startswith('http'):
          if not form['action'].startswith('http'):
              if not form['action'].startswith('/'):
                  Form['internals'].append(hostname+'/'+form['action']) 
              elif form['action'] in Null_format or form['action'] == 'about:blank':
                  Form['null'].append(form['action'])  
              else:
                  Form['internals'].append(hostname+form['action'])   
      else:
          Form['externals'].append(form['action'])
  
  def sf(hostname, Form):
    if len(Form['null'])==0:
        return 1
    elif len(Form['null'])>0:
        return 0
    else:
      return -1
  
  return(sf(hostname, Form))

In [22]:
sfh('https://stackoverflow.com/questions/59020008/how-to-import-functions-of-a-jupyter-notebook-into-another-jupyter-notebook-in-g')

1

### Submitting to emial

In [23]:
def sub_email(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
  
  Form = {'internals':[], 'externals':[], 'null':[]}
  def findForm(Form, domain, hostname):
    for form in soup.findAll('form', action=True):
      dots = [x.start(0) for x in re.finditer('\.', form['action'])]
      if hostname in form['action'] or domain in form['action'] or len(dots) == 1 or not form['action'].startswith('http'):
          if not form['action'].startswith('http'):
              if not form['action'].startswith('/'):
                  Form['internals'].append(hostname+'/'+form['action']) 
              elif form['action'] in Null_format or form['action'] == 'about:blank':
                  Form['null'].append(form['action'])  
              else:
                  Form['internals'].append(hostname+form['action'])   
      else:
          Form['externals'].append(form['action'])
  
  def submitting_to_email(Form):
    # print(Form)
    for form in (Form['internals'] + Form['externals']):
        #print('inside for')
        if "mailto:" in form or "mail()" in form:
            return 1
        else:
            return -1
    return 1
  
  return(submitting_to_email(Form))

In [24]:
# print('ans is ',  sub_email('https://www.xiaoji.com/'))

### On Mouse-over

In [25]:
def mouse_over(url):
  from urllib.parse import urlparse,urlencode,urlsplit, urlunsplit
  from bs4 import BeautifulSoup
  import requests
  import re
  !pip install tldextract
  import tldextract
  Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
               "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]
  
  def is_URL_accessible(url):
    page = None
    try:
        page = requests.get(url, timeout=5)   
    except:
        parsed = urlparse(url)
        url = parsed.scheme+'://'+parsed.netloc
        if not parsed.netloc.startswith('www'):
            url = parsed.scheme+'://www.'+parsed.netloc
            try:
                page = requests.get(url, timeout=5)
            except:
                page = None
                pass
    if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
        return True, url, page
    else:
        return False, None, None
    
  state, iurl, page = is_URL_accessible(url)

  def get_domain(url):
      o = urlsplit(url)
      return o.hostname, tldextract.extract(url).domain, o.path
    
  if state:
        content = page.content
        hostname, domain, path = get_domain(url)
  
  def onmouseover(content):
    if 'onmouseover="window.status=' in str(content).lower().replace(" ",""):
        return 1
    else:
        return -1
  return(onmouseover(content.decode('latin-1')))

In [26]:
mouse_over('https://www.xiaoji.com/')

-1

### Right Click

In [55]:
from urllib.parse import urlparse, urlsplit
from bs4 import BeautifulSoup
import requests
import re
import tldextract

def right_click(url):
    Null_format = ["", "#", "#nothing", "#doesnotexist", "#null", "#void", "#whatever",
                   "#content", "javascript::void(0)", "javascript::void(0);", "javascript::;", "javascript"]

    def is_URL_accessible(url):
        page = None
        try:
            page = requests.get(url, timeout=5)   
        except:
            parsed = urlparse(url)
            url = parsed.scheme+'://'+parsed.netloc
            if not parsed.netloc.startswith('www'):
                url = parsed.scheme+'://www.'+parsed.netloc
                try:
                    page = requests.get(url, timeout=5)
                except:
                    page = None
                    pass
        if page and page.status_code == 200 and page.content not in ["b''", "b' '"]:
            return True, url, page
        else:
            return False, None, None

    state, iurl, page = is_URL_accessible(url)

    if state:
        content = page.content
        
        def get_domain(url):
            o = urlsplit(url)
            return o.hostname, tldextract.extract(url).domain, o.path

        hostname, domain, path = get_domain(url)

        def right_clic(content):
            if re.findall(r"event.button ?== ?2", content.decode('latin-1')):
                return 1
            else:
                return -1

        return right_clic(content)
    else:
        return -1




In [56]:

print(right_click('http://walletconnectbits.com/'))

-1


Domain Age

In [33]:
import json
import requests

def domain_age(domain):
    url = domain.split("//")[-1].split("/")[0].split('?')[0]
    show = "https://input.payapi.io/v1/api/fraud/domain/age/" + url

    retries = 3  # Number of retries
    timeout = 10  # Timeout in seconds

    for _ in range(retries):
        try:
            r = requests.get(show, timeout=timeout)
            if r.status_code == 200:
                data = r.text
                jsonToPython = json.loads(data)
                result = jsonToPython['result']
                if result / 30 >= 6:
                    return -1
                else:
                    return 1
            else:
                return -1
        except requests.exceptions.Timeout:
            # Retry on timeout
            continue
        except Exception as e:
            print(f"Error occurred: {e}")
            return -1

    # Return -1 if all retries fail
    return -1




In [34]:
print(domain_age('http://walletconnectbits.com/'))

-1


### DNS Record

In [59]:
def dns_record(domain):
    !pip install dnspython
    import dns.resolver
    try:
        nameservers = dns.resolver.resolve(domain,'NS')
        if len(nameservers) > 0:
            #print('len is', nameservers)
            return -1
        else:
            return 1
    except:
        return 1

In [60]:
print(dns_record('http://postdebanks.com/DIE/POST/diepost/'))

1


###Website Traffic 

In [63]:
from urllib.parse import quote
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import URLError

def web_traffic(url):
    try:
        # Filling the whitespaces in the URL if any
        url = quote(url)
        rank = BeautifulSoup(urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        rank = int(rank)
        if rank <= 100000:
            return -1
        elif rank > 100000:
            return 0
    except (TypeError, KeyError, URLError):
        # Handle exceptions gracefully
        return 1

# Test the function
print(web_traffic('http://postdebanks.com/DIE/POST/diepost/'))


1


In [64]:
# Test the function
print(web_traffic('http://postdebanks.com/DIE/POST/diepost/'))

1


### Page Rank

In [65]:
def page_rank(domain):
    import requests
    key = '8o0gg0804g4k0gwkk4oocws04oc0sg88gg844o4k'
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + str(domain)
    try:
        request = requests.get(url, headers={'API-OPR': key})
        result = request.json()
        result = result['response'][0]['page_rank_integer']
        if result / 10 < 0.2 :
            return 1
        else:
            return -1
    except:
        return 1

In [66]:
print(page_rank('http://postdebanks.com/DIE/POST/diepost/'))

1


### Google Index

In [67]:
def google_index(url):
    from urllib.parse import urlencode
    from bs4 import BeautifulSoup
    import requests
    #time.sleep(.6)
    user_agent =  'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
    headers = {'User-Agent' : user_agent}
    query = {'q': 'site:' + url}
    google = "https://www.google.com/search?" + urlencode(query)
    data = requests.get(google, headers=headers)
    data.encoding = 'ISO-8859-1'
    soup = BeautifulSoup(str(data.content), "html.parser")
    try:
      if 'Our systems have detected unusual traffic from your computer network.' in str(soup):
        return -1
      check = soup.find(id="rso").find("div").find("div").find("a")
      #print(soup.prettify())
      if check and check['href']:
        return -1
      else:
        return 1
        
    except AttributeError:
        return 1

In [68]:
print(google_index('http://walletconnectbits.com/'))

1


### Links pointing to page (No of Hyperlinks)

In [73]:
from bs4 import BeautifulSoup
from collections import Counter
import requests
from requests.exceptions import RequestException

def urlsCount(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        soup = BeautifulSoup(response.text, "html.parser")
        foundUrls = Counter([link["href"] for link in soup.find_all("a", href=lambda href: href and not href.startswith("#"))])
        count = len(foundUrls)
        if count == 0:
            return 1
        elif 0 < count <= 2:
            return 0
        else:
            return -1
    except RequestException as e:
        print(f"Error: {e}")
        return 1  # Return a default value or handle the error as needed



In [74]:
# Test the function
print(urlsCount('https://www.hokabutypolska.pl/'))


Error: HTTPSConnectionPool(host='www.hokabutypolska.pl', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002749BA7CFD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
1


### statistical report

In [75]:
def statistical_report(url, domain):
    import re
    import socket
    url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
    try:
        ip_address=socket.gethostbyname(domain)
        ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|'
                           '107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|'
                           '118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|'
                           '216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|'
                           '34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|'
                           '216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)
        if url_match or ip_match:
            return 1
        else:
            return -1
    except:
        return 1

In [76]:
print(statistical_report('http://postdebanks.com/DIE/POST/diepost/', 'postdebanks.com'))

1


### Input features

In [77]:
def getInput(url):
  from urllib.parse import urlparse
  domain = urlparse(url).netloc
  input = []
  print(domain)
  input.append(having_ip_address(url))
  input.append(URL_Length(url))
  input.append(haveAtSign(url))
  input.append(prefixSuffix(url))
  input.append(sub_domain_count(url))
  input.append(sslVerify(url))
  input.append(port(domain))
  input.append(request_url(url))
  input.append(url_anchor(url))
  input.append(links_tag(url))
  input.append(sfh(url))
  input.append(sub_email(url))
  input.append(mouse_over(url))
  input.append(right_click(url))
  input.append(domain_age(url))
  input.append(dns_record(url))
  input.append(web_traffic(url))
  input.append(page_rank(domain))
  input.append(google_index(url))
  input.append(urlsCount(url))
  input.append(statistical_report(url, domain))
  return (input)

In [ ]:
input = getInput('https://www.youtube.com/')

In [89]:
input

[-1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1]

In [90]:
[input]

[[-1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1]]

In [91]:
import pickle

filename = r"finalized_model.sav"
loaded_model = pickle.load(open(filename, 'rb'))


In [92]:
import warnings
warnings.filterwarnings("ignore")

In [93]:
result = loaded_model.predict([input])
if result == -1:
  print("A legitimate website")
else:
  print("A Phishing website!!")

A legitimate website
